# 1.Data Import and Combination

In [2]:
import pandas as pd
import gensim as gs
import nltk
import re
from nltk.corpus import stopwords
import sklearn as sk
import numpy as np
import pickle
from tqdm import tqdm

## 1.1.import dataset

In [3]:
train_path="DataSet\\train.txt"
test_path="DataSet\\test.txt"
train_df = pd.read_csv(train_path,sep="\t",index_col=0)
test_df=pd.read_csv(test_path,sep="\t",index_col=0)

In [4]:
train_df

,turn1,turn2,turn3,label
id,,,,
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others


In [5]:
test_df

,turn1,turn2,turn3,label
id,,,,
0,Hmm,What does your bio mean?,I don’t have any bio,others
1,What you like,very little things,Ok,others
2,Yes,How so?,I want to fuck babu,others
3,what did you guess,what what,fuck,others
4,We ?,of course we will!,What gender movies you like??,others
...,...,...,...,...
5504,Not youuu,I also didn't not not.,How to calll,others
5505,Welcome,"Why, thank you.","I don't know, you tell",others
5506,Yes,IF ONLY I COULD AFFORD THIS,How are you,others


## 1.2.unify the variables for change

In [6]:
# now when I copy the code for another dataset, I can easily change the special name of the dataset for adjusting the process and storage
version_change = "tfidf"
path_change = "combine"

In [7]:
# emo_path="DataSet\\emo_collection_{b}.csv".format(b=version_change)
# emo_df=pd.read_csv(emo_path,sep="|")

In [8]:
# emo_df

## 1.3.get label and turn into category

In [9]:
# set the label into categorical number:
# others:0; happy:1,angry:3;sad:2;

class Label_format:
    def __init__(self,df):
        self.df=df
        
    def get_y(self):
        y = self.df.iloc[:,-1].values
        return y
    
    def label_category(self,y):
        change_list = list()
        emotion = {"others":0,"happy":1,"sad":2,"angry":3}
        for each in y:
            change_list.append(emotion[each])
        change_array=np.array(change_list)
        return change_array
    
    def save_file(self,data,name):
        pickle_file = open('DataSet\\'+name,mode='wb')
        pickle.dump(data,pickle_file)
        pickle_file.close()

In [10]:
# set y label, and we make X later on.
# for traindata
lf = Label_format(train_df)
y = lf.get_y()
y = lf.label_category(y)
save_path = "{a}_y.pkl".format(a='train')
lf.save_file(y,save_path)

In [11]:
# set y label, and we make X later on.
# for testdata
lf = Label_format(test_df)
y = lf.get_y()
y = lf.label_category(y)
save_path = "{a}_y.pkl".format(a='test')
lf.save_file(y,save_path)

In [12]:
y

array([0, 0, 0, ..., 0, 0, 0])

## 1.4. class for combination

In [13]:
# because we use pre-trained embeddings, we can't directly use stemming, remove stopwords or remove punctuations.
# we need to turn the data as much as possible close to the pre-trained embedding.
def basic_clean_text(input_text):
    'clean text function'
    #clean emoji, punctions
    input_text = re.sub(r'[^a-zA-Z]',repl=" ",string=input_text)
    #lower the words
    input_text =input_text.lower()
    #stemming the words
    list_text=input_text.split()
    ps=nltk.stem.porter.PorterStemmer()
    list_text_out=list()
    for word in list_text:
        if word in set(stopwords.words('english')):
            continue
        list_text_out.append(ps.stem(word))
    input_text=" ".join(list_text_out) 
    return input_text

def combine_method(df):
    turn1_corpus= list(df["turn1"])
    turn2_corpus= list(df["turn2"])
    turn3_corpus= list(df["turn3"])
    
    # use text_content as dict for storing the data of 
    text_content={"turn1":turn1_corpus,"turn2":turn2_corpus,"turn3":turn3_corpus}
    
    # because we need all the sentences including sentences in turn1, turn2, turn3 
    # to predict the emtion situation, therefore we need to combine them together
    length = len(text_content["turn1"])
    text_all_content = list()
    for i in tqdm(range(length)):
        text_row_all = text_content["turn1"][i] +" " + text_content["turn2"][i] +" "+ text_content["turn3"][i]
        text_all_content.append(text_row_all)
    text_all_content = {"all":text_all_content}
    
    return text_content,text_all_content

In [14]:
text_3,text_all = combine_method(train_df)

100%|███████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 1080016.81it/s]


In [15]:
text_3_test, text_all_test = combine_method(test_df)

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 920391.19it/s]


In [16]:
text_all

{'all': ["Don't worry  I'm girl hmm how do I know if you are What's ur name?",
  'When did I? saw many times i think -_- No. I never saw you',
  'By by Google Chrome Where you live',
  'U r ridiculous I might be ridiculous but I am telling the truth. U little disgusting whore',
  'Just for time pass wt do u do 4 a living then Maybe',
  "I'm a dog person youre so rude Whaaaat why",
  'So whatsup Nothing much. Sitting sipping and watching TV. How abt u? What are you watching on tv?',
  'Ok ok im back!! So, how are u',
  'Really? really really really really really Y saying so many times...i can hear you',
  'Bay in the bay 😘 love you',
  'I hate my boyfriend you got a boyfriend? Yes',
  'I will do night. Alright. Keep me in loop. Not giving WhatsApp no.',
  'Sure go ahead Many thanks once again! Love you too',
  "Bad Bad bad! That's the bad kind of bad. I have no gf",
  'Ok get it...... I made it an option Ok',
  "Money money and lots of money😍😍 I need to get it tailored but I'm in love w

## 1.5. Main Function

In [17]:
def fit_transform(input_dict,method):
    output_dict=dict()
    for key in input_dict.keys():
        processed_texts = list()
        for text in tqdm(input_dict[key]):
            processed_single_text = eval(method)(text)
            processed_texts.append(processed_single_text)
        
        output_dict[key]=processed_texts
    
    return output_dict

# 2. Preprocessing for cleaning the data

# 2.1. Lower the words

In [18]:
# lower the words
def lower_text(input_text):
    #lower the words
    input_text =input_text.lower()
    return input_text

In [19]:
text_3_1 = fit_transform(text_3,"lower_text")
text_all_1 = fit_transform(text_all,"lower_text")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 181081.92it/s]


In [20]:
text_3_test_1 = fit_transform(text_3_test,"lower_text")
text_all_test_1 = fit_transform(text_all_test,"lower_text")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 178180.30it/s]


## 2.2. Seperate the adhered words with space and remove redundant emoji, emoticons
 - e.g. "you?" ? and you are adhered together, which needs to be split
 - e.g. "yess i am crazyyy😂😂" I also need to split emoji from the word "crazyyy" as "i am crazyyy 😂 😂 "
 - and some emoji, emoticons can't be found in the emo_df, therefore, I delete them since it's impossible to explain them in word2vec

In [21]:
# functions for extracting emoji, emoticons and punctuations
def extract_emoji(input_text):
    emoji_list = re.findall(r'(?:[^a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
    return emoji_list
    
def extract_emoticon(input_text):
    emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
    return emoticon_list

def extract_punctuation(input_text):
    'here I use for loop to split the double punctuations into single ones'
    punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
    result_list=list()
    for each in punctuation_list:
        list_split=[each]
        if each !="...":
            list_split=list(each)
        for punc in list_split:
            result_list.append(punc)
    return result_list

# functions for assigning emoji, emoticons and punctuations space from each other
def assign_space(input_text):
    #emoji
    input_text = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',' ',input_text)
    #emoticon
    input_text = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',' ',input_text)
    #punctuation
    input_text = re.sub(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',' \g<0> ' ,input_text)
    #replace the redundant space
    input_text = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
    input_text = re.sub(r'  +',r' ',input_text)
    return input_text

In [22]:
text_3_2 = fit_transform(text_3_1,"assign_space")
text_all_2 = fit_transform(text_all_1,"assign_space")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:01<00:00, 27591.82it/s]


In [23]:
text_3_test_2 = fit_transform(text_3_test_1,"assign_space")
text_all_test_2 = fit_transform(text_all_test_1,"assign_space")

100%|███████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 27458.71it/s]


# 3. Preprocessing: Correct the words in sentence
## 3.1. Correct common words

In [24]:
# turn don't, doesn't, can't, couldn't, shouldn't, didn't, isn't, wasn't, weren't, aren't, won't, shan't, 's, 're
# as do not, do not, can not, could not, should not, did not, is not, was not, were not, are not, will not, shall not, 's, are

# turn r , u, bt, n, 
# as are, you ,but, and
cor_dict= {
    "don ' t":      "do not",
    "doesn ' t":    "do not",
    "can ' t":      "can not",
    "couldn ' t":   "could not",
    "shouldn ' t":  "should not",
    "didn ' t":     "did not",
    "isn ' t":      "is not",
    "wasn ' t":     "was not",
    "weren ' t":    "were not",
    "aren ' t":     "are not",
    "won ' t":      "will not",
    "shan ' t":     "shall not",    
    #"\' re ":       "are ",
    #" re ":         "are",
    "\' m ":        "am ",
    "\' ll ":       "will ",
    #" \' s ":       " \'s "

    }

In [25]:
def correct_words_common(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = input_text
    for each_key in cor_dict.keys():
        if each_key in input_text:
            corrected_sentence = corrected_sentence.replace(each_key,cor_dict[each_key])
    return corrected_sentence


In [26]:
text_3_3 = fit_transform(text_3_2,"correct_words_common")
text_all_3 = fit_transform(text_all_2,"correct_words_common")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 133219.11it/s]


In [27]:
text_3_test_3 = fit_transform(text_3_test_2,"correct_words_common")
text_all_test_3 = fit_transform(text_all_test_2,"correct_words_common")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 134747.82it/s]


## 3.2. Correct missing characters
- e.g. "hmmm i can talk no w", here "now" is missing "w".

In [28]:
def correct_words_combin(input_text):
    corrected_sentence = input_text
    corrected_sentence = re.sub(r' ([a-z]+) ([bcdefghjklnopqrstvwxz]) ',r' \g<1>\g<2> ',corrected_sentence)
    return corrected_sentence

In [29]:
testset = {"turn1":["I ' m go d , and I wasn ' t player wai t ", "hhh you ' re kidding m e "]}
print(fit_transform(testset,"correct_words_combin"))

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2006.36it/s]

{'turn1': ["I ' m god , and I wasn ' t player wait ", "hhh you ' re kidding me "]}


In [30]:
text_3_4 = fit_transform(text_3_3,"correct_words_combin")
text_all_4 = fit_transform(text_all_3,"correct_words_combin")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 93324.68it/s]


In [31]:
text_3_test_4 = fit_transform(text_3_test_3,"correct_words_combin")
text_all_test_4 = fit_transform(text_all_test_3,"correct_words_combin")

100%|███████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 83631.95it/s]


## 3.3. Remove punctuations

In [32]:
def remove_punc(input_text):
    new_sentence = input_text
    new_sentence = re.sub(r'[^ a-zA-Z]',"",new_sentence).strip()
    return new_sentence

In [33]:
text_3_5 = fit_transform(text_3_4,"remove_punc")
text_all_5 = fit_transform(text_all_4,"remove_punc")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 127578.89it/s]


In [34]:
len(text_all_5["all"])

30160

In [35]:
text_3_test_5 = fit_transform(text_3_test_4,"remove_punc")
text_all_test_5 = fit_transform(text_all_test_4,"remove_punc")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 131476.97it/s]


In [36]:
len(text_all_test_5["all"])

5509

## 3.4.  Remove stopwords made handcrafted

In [37]:
print(stopwords.words('english')) # which is not suitable here, coz e.g. in emotion analysis, not happy is opposite from happy.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [38]:
stopwords_byhand = []
# since combination of words can make the sentiment totally different
# e.g. "wasn't happy" is more likely to be sad, not happy.
# e.g. "but but but ...." could mean frustrating emotion, because someone wants to explain sth

In [39]:
def remove_stopwords(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        if word in stopwords_byhand:
            continue
        else:
            new_words.append(word)
    sentence = " ".join(new_words)
  
    return sentence

In [45]:
text_3_6 = fit_transform(text_3_5,"remove_stopwords")
text_all_6 = fit_transform(text_all_5,"remove_stopwords")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 116309.95it/s]


In [48]:
text_3_test_6 = fit_transform(text_3_test_5,"remove_stopwords")
text_all_test_6 = fit_transform(text_all_test_5,"remove_stopwords")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 122527.83it/s]


In [49]:
len(text_3_test_6["turn1"])

5509

# 4. Preprocessing: Correction of Incorrect Spell and Stemming
 - e.g. "nt" is actually "not", "noo" is actually "no"
 - in order to do so, here i use library called pyspellchecker

In [50]:
from spellchecker import SpellChecker
from nltk.stem.porter import PorterStemmer
spell = SpellChecker()
ps = PorterStemmer()
def correction_stemming(input_text):
    words = input_text.split()
    new_words=list()
    for word in words:
        word = spell.correction(word)
        word = ps.stem(word)
        new_words.append(word)
    sentence = " ".join(new_words)
    return sentence


In [51]:
# test function
example = "i like playing guitarrr"
print(correction_stemming(example))

i like play guitar


In [52]:
text_3_7 = fit_transform(text_3_6,"correction_stemming")
text_all_7 = fit_transform(text_all_6,"correction_stemming")

100%|████████████████████████████████████████████████████████████████████████████| 30160/30160 [23:53<00:00, 21.04it/s]


In [53]:
text_3_test_7 = fit_transform(text_3_test_6,"correction_stemming")
text_all_test_7 = fit_transform(text_all_test_6,"correction_stemming")

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [04:55<00:00, 18.63it/s]


# 4. check data format 
- for further preprocessing based on pre-trained embedding resource

In [54]:
def distinct_words(dialogs):
    vocab = dict()
    for each_dialog in tqdm(dialogs):
        words_list = each_dialog.split(sep=" ")
        for word in words_list:
            if word == '':
                continue
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    
    return vocab

In [55]:
vocab = distinct_words(text_all["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version original",b=len(vocab)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 187830.68it/s]

the length of distinct vocabulary of the version original is 33024


In [56]:
vocab1 = distinct_words(text_all_1["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 1",b=len(vocab1)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 182171.96it/s]

the length of distinct vocabulary of the version 1 is 27889


In [57]:
vocab2 = distinct_words(text_all_2["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 2",b=len(vocab2)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 176761.69it/s]

the number of distinct vocabulary of the version 2 of data is 15605


In [58]:
vocab3 = distinct_words(text_all_3["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 3",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 192616.93it/s]

the number of distinct vocabulary of the version 3 of data is 15601


In [59]:
vocab4 = distinct_words(text_all_4["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 4",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 241883.45it/s]

the number of distinct vocabulary of the version 4 of data is 15601


In [60]:
vocab5 = distinct_words(text_all_5["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 5",b=len(vocab5)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 256278.22it/s]

the number of distinct vocabulary of the version 5 of data is 15248


In [61]:
vocab6 = distinct_words(text_all_6["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 6",b=len(vocab6)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 258432.67it/s]

the number of distinct vocabulary of the version 6 of data is 15248


In [62]:
vocab7 = distinct_words(text_all_7["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 7",b=len(vocab7)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 260487.34it/s]

the number of distinct vocabulary of the version 7 of data is 8399


# 5.TF-IDF transformation

In [63]:
def computeTF(word,text):
    words_list = text.split()
    no_all = len(words_list)
    no_word = 0
    try:
        words_list.remove(word)
        no_word +=1
    except:
        pass
    tf = no_word / no_all
    return tf

In [64]:
# test function
list1="hhh I don't know exactly , so?"
print(computeTF("don't",list1))

0.14285714285714285


In [65]:
def computeDenominatorIDF(text_all):
    doc_word=dict()
    for text in tqdm(text_all):
        words_list = text.split(sep=" ")
        distinct_list = list(set(words_list)) # delete duplication
        for word in distinct_list:
            if word =='':
                continue
            try:
                doc_word[word] +=1 # calculate how many document own such word
            except KeyError:
                doc_word[word] = 1
    
    return doc_word

In [66]:
# test function
list1= ["what do you like","i like very little thing"]
print(computeDenominatorIDF(list1))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

{'like': 2, 'what': 1, 'you': 1, 'do': 1, 'very': 1, 'thing': 1, 'i': 1, 'little': 1}


In [67]:
def computeIDF(text_all):
    words_idf = dict()
    no_doc = len(text_all)
    doc_word = computeDenominatorIDF(text_all)
    for word in doc_word.keys():
        words_idf[word] = no_doc / (doc_word[word])
    
    columns_list = list(words_idf.keys())
    
    return words_idf, columns_list

In [68]:
# test function
list1= ["what do you like","i like very little thing"]
print(computeIDF(list1))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

({'like': 1.0, 'what': 2.0, 'you': 2.0, 'do': 2.0, 'very': 2.0, 'thing': 2.0, 'i': 2.0, 'little': 2.0}, ['like', 'what', 'you', 'do', 'very', 'thing', 'i', 'little'])


In [69]:
def computeTF_IDF(text_all,words_idf,columns_list):
    tfidf_text_all = list()
    for text in tqdm(text_all):
        tfidf_text=dict()
        words_list = text.split()
        distinct_list = list(set(words_list))# delete duplication
        for word in distinct_list:
            if word =='':
                continue
            try:
                idf = words_idf[word]
                tf = computeTF(word,text)
                tf_idf = tf * idf
                tfidf_text[word]=tf_idf
            except KeyError:
                continue
        
        row_vector = list()
        for word in columns_list:
            if word in distinct_list:
                row_vector=row_vector + [tfidf_text[word]]
            else:
                list_empty=[0.0]
                row_vector=row_vector + list_empty
        
        tfidf_text_all.append(row_vector)
    
    return tfidf_text_all

In [70]:
# firstly, i need to use train data to calculate IDF, 
# coz test data should be granted as unseen data for now, and transfer them into tf-idf vecotrs based on train data
words_idf, columns_list = computeIDF(text_all_7["all"])

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 195716.84it/s]


In [71]:
len(columns_list)

8399

## 5.1.process the 3 turn dialogue

In [72]:
# firstly get 1st turn and 3rd turn together, and get rid of 2nd turn dialogue
def delete_2ndturn(text_3turn):
    length = len(text_3turn["turn1"])
    new_text = list()
    for i in range(length):
        text1=text_3turn["turn1"][i]+" "+text_3turn["turn1"][i]
        text2=""
        text3=text_3turn["turn3"][i]+" "+text_3turn["turn3"][i]
        new_text.append((text1+ " "+ text3))
    return new_text

In [73]:
text_3_final_no2 = delete_2ndturn(text_3_7)
text_3_finaltest_no2 = delete_2ndturn(text_3_test_7)
tfidf_text_3 = computeTF_IDF(text_3_final_no2,words_idf,columns_list)
tfidf_text_test_3 = computeTF_IDF(text_3_finaltest_no2,words_idf,columns_list)
# then form the dataframe based on the transfered data
train_df_tfidf_no2 = pd.DataFrame(data=tfidf_text_3, columns = columns_list)
test_df_tfidf_no2 = pd.DataFrame(data=tfidf_text_test_3, columns = columns_list)

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [18:37<00:00,  4.93it/s]


In [74]:
# secondly get 1st turn and 3rd turn with higher importance by multiply the words with 2 times
def double_1st3rdturn(text_3turn):
    length = len(text_3turn["turn1"])
    new_text = list()
    for i in range(length):
        text1=text_3turn["turn1"][i]+" "+text_3turn["turn1"][i]
        text2=text_3turn["turn2"][i]
        text3=text_3turn["turn3"][i]+" "+text_3turn["turn3"][i]
        new_text.append((text1+" "+ text2+ " "+ text3))
    return new_text

In [75]:
text_3_final_double13 = double_1st3rdturn(text_3_7)
text_3_finaltest_double13 = double_1st3rdturn(text_3_test_7)
tfidf_text_3 = computeTF_IDF(text_3_final_no2,words_idf,columns_list)
tfidf_text_test_3 = computeTF_IDF(text_3_finaltest_no2,words_idf,columns_list)
# then form the dataframe based on the transfered data
train_df_tfidf_double13 = pd.DataFrame(data=tfidf_text_3, columns = columns_list)
test_df_tfidf_double13 = pd.DataFrame(data=tfidf_text_test_3, columns = columns_list)

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [09:53<00:00,  9.27it/s]


In [92]:
test_df_tfidf_double13.head()

,if,what,hmm,worri,know,do,s,not,am,are,...,heheheheh,roadsid,chiranjeevi,pawankalyan,drone,salaman,conphyooj,grasshopp,freeeeeeeeeeeeee,dilut
0,0.0,0.000000,9.70399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.600414,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.480331,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.400276,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# thirdly give 1st turn higher importance by multiplied by 2 and give 3td turn highest importance by multiply the words by 3
def times3_3rdturn(text_3turn):
    length = len(text_3turn["turn1"])
    new_text = list()
    for i in range(length):
        text1=text_3turn["turn1"][i]+" "+text_3turn["turn1"][i]
        text2=text_3turn["turn2"][i]
        text3=text_3turn["turn3"][i]+" "+text_3turn["turn3"][i]+" "+text_3turn["turn3"][i]
        new_text.append((text1+" "+ text2+ " "+ text3))
    return new_text

In [77]:
text_3_final_time3 = times3_3rdturn(text_3_7)
text_3_finaltest_time3 = times3_3rdturn(text_3_test_7)
tfidf_text_3 = computeTF_IDF(text_3_final_no2,words_idf,columns_list)
tfidf_text_test_3 = computeTF_IDF(text_3_finaltest_no2,words_idf,columns_list)
# then form the dataframe based on the transfered data
train_df_tfidf_time3 = pd.DataFrame(data=tfidf_text_3, columns = columns_list)
test_df_tfidf_time3 = pd.DataFrame(data=tfidf_text_test_3, columns = columns_list)

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [10:17<00:00,  8.92it/s]


In [89]:
# thirdly give 1st turn higher importance by multiplied by 2 and give 3td turn highest importance by multiply the words by 3
def delete2nd_bigger3(text_3turn):
    length = len(text_3turn["turn1"])
    new_text = list()
    for i in range(length):
        text1=text_3turn["turn1"][i]
        text2=text_3turn["turn2"][i]
        text3=text_3turn["turn3"][i]+" "+text_3turn["turn3"][i]
        new_text.append((text1+ " "+ text3))
    return new_text

In [90]:
text_3_final_del2big3 = delete2nd_bigger3(text_3_7)
text_3_finaltest_del2big3 = delete2nd_bigger3(text_3_test_7)
tfidf_text_3 = computeTF_IDF(text_3_final_no2,words_idf,columns_list)
tfidf_text_test_3 = computeTF_IDF(text_3_finaltest_no2,words_idf,columns_list)
# then form the dataframe based on the transfered data
train_df_tfidf_del2big3 = pd.DataFrame(data=tfidf_text_3, columns = columns_list)
test_df_tfidf_del2big3 = pd.DataFrame(data=tfidf_text_test_3, columns = columns_list)

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [11:40<00:00,  7.87it/s]


## 5.2. process the overall dialogue

In [78]:
# train data transformation
tfidf_text_all = computeTF_IDF(text_all_7["all"],words_idf,columns_list)

100%|██████████████████████████████████████████████████████████████████████████| 30160/30160 [1:00:24<00:00,  8.32it/s]


In [79]:
# test data transformation
tfidf_text_test_all = computeTF_IDF(text_all_test_7["all"],words_idf,columns_list)

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [10:55<00:00,  8.40it/s]


In [80]:
# then form the dataframe based on the transfered data
train_df_tfidf = pd.DataFrame(data=tfidf_text_all, columns = columns_list)

In [81]:
train_df_tfidf.head()

,if,what,hmm,worri,know,do,s,not,am,are,...,heheheheh,roadsid,chiranjeevi,pawankalyan,drone,salaman,conphyooj,grasshopp,freeeeeeeeeeeeee,dilut
0,1.811411,0.266851,6.469326,13.622403,0.66649,0.289638,0.563212,0.213284,0.295565,0.31208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.332510,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.434457,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
# test data
test_df_tfidf = pd.DataFrame(data=tfidf_text_test_all, columns = columns_list)

In [83]:
test_df_tfidf.head()

,if,what,hmm,worri,know,do,s,not,am,are,...,heheheheh,roadsid,chiranjeevi,pawankalyan,drone,salaman,conphyooj,grasshopp,freeeeeeeeeeeeee,dilut
0,0.0,0.436665,10.586171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.686188,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.686188,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.480331,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 6. data storage

## 6.1.save the 3 turn processed data

In [84]:
train_df_tfidf_no2.to_csv("DataSet\\train_df_tfidf_no2.csv",sep=",",index=False)
test_df_tfidf_no2.to_csv("DataSet\\test_df_tfidf_no2.csv",sep=",",index=False)

In [87]:
train_df_tfidf_double13.to_csv("DataSet\\train_df_tfidf_double3.csv",sep=",",index=False)
test_df_tfidf_double13.to_csv("DataSet\\test_df_tfidf_double3.csv",sep=",",index=False)

In [88]:
train_df_tfidf_time3.to_csv("DataSet\\train_df_tfidf_time3.csv",sep=",",index=False)
test_df_tfidf_time3.to_csv("DataSet\\test_df_tfidf_time3.csv",sep=",",index=False)

In [91]:
train_df_tfidf_del2big3.to_csv("DataSet\\train_df_tfidf_del2big3.csv",sep=",",index=False)
test_df_tfidf_del2big3.to_csv("DataSet\\test_df_tfidf_del2big3.csv",sep=",",index=False)

## 6.2. save the overall data

In [ ]:
train_df_tfidf.to_csv("DataSet\\train_df_tfidf.csv",sep=",",index=False)

In [ ]:
test_df_tfidf.to_csv("DataSet\\test_df_tfidf.csv",sep=",",index=False)